In [24]:
import numpy as np
import pandas as pd

In [22]:
df = pd.read_json("data/dataset.json")

In [27]:
df.to_numpy()

array([41263,
       list([[41.11, 21.31], [41.41, 22.63], [42.1, 24.02], [41.93, 25.65], [41.38, 26.92]]),
       list([[43.68, 21.93], [43.35, 25.18], [43.7, 26.67], [43.9, 32.63]]),
       list([41.56, 23.92]), 2021090900, 97, 6, 'DAL', 'right',
       'ball_snap', list([43.3, 18.89]), 6, 38, 'Pass Rush', 'LEO', 'TB',
       1, 3, 2, '13:33:00', 0, 0, 43, '1 RB, 1 TE, 3 WR', 6, 'SHOTGUN'],
      dtype=object)